DL9 (gr.1) Learning binarized neural networks using the approach given in [M.Courbariaux]. Comparison of the Adam and RMSProp versions of the SGD algorithm proposed therein. The comparison should be performed on the example of learning LSTM model discussed in [XuanLiu]. Project for 3 students.

In [215]:
import keras
import numpy as np
from tensorflow.keras.layers import LSTM
import tensorflow.compat.v2 as tf
import BinaryLSTM
import importlib
importlib.reload(BinaryLSTM)
import larq
from larq.quantizers import ste_sign

In [2]:
inputs = tf.random.normal([32, 10, 8])
lstm_b = tf.keras.layers.LSTM(4, kernel_constraint='weight_clip', activation='ste_sign', kernel_regularizer='ste_sign', recurrent_activation='ste_sign', return_sequences=True)
lstm_k = tf.keras.layers.LSTM(4, kernel_constraint='weight_clip')
lstm_a = tf.keras.layers.LSTM(4, activation='ste_sign')
lstm = tf.keras.layers.LSTM(4)
output = lstm_b(inputs)
print(output.shape)

(32, 10, 4)


In [3]:
inputs = keras.Input(shape=(10,8))
b = BinaryLSTM.BinaryLSTM(16)
outputs = b(inputs)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
model.compile(loss='MSE', optimizer='adam')
outputTensor = model.output
listOfVariableTensors = model.trainable_weights
gradients = tf.GradientTape(outputTensor, listOfVariableTensors)

In [4]:
inp = tf.random.normal([32, 64])
z0_t, z1_t, z2_t, z3_t = tf.split(inp, 4, axis=1)
out = tf.concat([z0_t, z1_t, z2_t, z3_t], axis=1)
np.all(tf.equal(inp, out))
print(tf.math.exp(tf.zeros((64,))).shape)

(64,)


In [5]:
inputs = tf.random.normal([32, 10, 8])
out1 = BinaryLSTM.BinaryLSTM(16)(inputs)
out1

<tf.Tensor: shape=(32, 16), dtype=float32, numpy=
array([[ 1.31765187e-01, -3.43377948e-01,  1.00309104e-01,
        -1.06411194e-02, -2.06324887e-02,  5.15699327e-01,
        -2.52300501e-02,  4.24111545e-01, -3.09819635e-02,
         1.91481322e-01, -4.21832502e-01,  4.34078068e-01,
        -2.31857985e-01,  4.36021119e-01, -4.01340187e-01,
         5.14631392e-03],
       [-1.56023979e-01,  1.92372441e-01, -4.30778712e-02,
         6.91143349e-02, -8.00477266e-02,  3.88880879e-01,
        -2.43197620e-01, -4.81225103e-01,  3.83519709e-01,
         1.47759736e-01,  1.38074249e-01,  3.66790071e-02,
        -3.77974212e-02,  3.87143224e-01,  1.71137407e-01,
         8.44763741e-02],
       [-1.19013190e-01,  2.56671757e-01, -2.18169972e-01,
         1.40715435e-01, -2.35081658e-01,  6.72194734e-02,
        -1.10272355e-01, -3.10102403e-01,  6.67158365e-01,
        -2.93036640e-01,  3.29724625e-02, -1.69427589e-01,
        -1.45049125e-01,  8.70607346e-02,  2.00964302e-01,
        -4.24

In [6]:
inputs = tf.random.normal([32, 10, 8])
with tf.GradientTape() as g:
  g.watch(inputs)
  y = BinaryLSTM.BinaryLSTM(16)(inputs)
dy_dx = g.gradient(y, inputs)
print(dy_dx)

tf.Tensor(
[[[ 1.15597472e-01 -8.75124242e-05 -7.81277642e-02 ...  2.59088352e-03
   -7.24104792e-02  2.54937857e-02]
  [ 9.58209038e-02  3.44602019e-02  3.74328978e-02 ... -8.70078877e-02
    1.36436224e-02  5.64901531e-03]
  [ 1.22292310e-01 -9.31118056e-03  1.20747834e-02 ...  2.50660405e-02
   -7.93883204e-02 -8.13484639e-02]
  ...
  [ 2.49430642e-01  1.75703168e-01  1.61776952e-02 ... -1.05282895e-01
   -8.95240903e-03  1.71650976e-01]
  [ 4.31200027e-01  9.59299803e-02 -9.19823796e-02 ...  1.86555460e-01
    3.42132092e-01 -2.42418051e-03]
  [ 2.79529780e-01 -8.88223648e-02 -3.95763427e-01 ...  1.24347493e-01
    3.53564531e-01  5.32158971e-01]]

 [[ 1.59680530e-01  1.62948035e-02 -1.17105402e-01 ...  4.18263674e-02
    1.48249678e-02  4.18227911e-03]
  [ 9.86256525e-02  3.87991816e-02 -2.99949758e-02 ...  5.27089424e-02
    3.77393104e-02 -1.38088875e-03]
  [ 9.62520689e-02  4.17630002e-02 -2.74989791e-02 ... -7.68199712e-02
   -9.04981941e-02  1.17259040e-01]
  ...
  [-5.935684

# This should deal with clipping weights

In [7]:
tf.random.set_seed(0)
inputs = tf.random.normal([4, 10, 8])
b = BinaryLSTM.BinaryLSTM(16, recurrent_constraint='weight_clip', kernel_constraint='weight_clip')
print(b(inputs))

tf.Tensor(
[[-4.89990830e-01  9.83730182e-02  2.30480749e-02 -2.44967937e-01
   2.97179490e-01  2.06419416e-02 -5.57305142e-02  1.74407512e-01
   1.18746206e-01  1.86846107e-02  1.38649479e-01 -1.06791966e-02
  -6.74881190e-02 -2.94934869e-01  2.47343898e-01  8.10989365e-02]
 [ 6.34068772e-02  2.06878960e-01  3.14615928e-02  2.28612646e-01
  -2.05055594e-01  1.60357296e-01  3.07867248e-02  2.53901891e-02
  -3.27846915e-01  3.26820016e-02 -3.60278130e-01 -5.42258024e-02
   2.29849026e-01 -4.09268029e-02 -5.29290992e-04  7.12332502e-03]
 [ 1.47974446e-01  1.04510248e-01  9.66634005e-02  4.22103740e-02
   1.00194653e-02 -2.62950629e-01 -5.18052280e-01 -1.81887016e-01
   4.40098792e-02 -8.89940411e-02  9.71260071e-02  1.94494382e-01
  -2.99092755e-03  7.44463652e-02 -6.14278391e-02 -1.78142667e-01]
 [ 8.21991190e-02 -8.79352614e-02  6.75147355e-01 -1.60533160e-01
  -7.53984079e-02  4.77349877e-01 -3.11969399e-01 -2.41199777e-01
  -4.34600651e-01  3.04286275e-02 -3.24144900e-01  5.58839262e

# This should deal with clipping gradients

In [8]:
# Create an optimizer with the desired parameters.
opt = tf.keras.optimizers.Adam(learning_rate=0.1, clipvalue=1)
var1, var2 = tf.Variable(1.0), tf.Variable(2.0)
# `loss` is a callable that takes no argument and returns the value
# to minimize.
loss = lambda: 3 * var1 * var1 + 2 * var2 * var2
# Call minimize to update the list of variables.
opt.minimize(loss, var_list=[var1, var2])
var1, var2

(<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=0.900001>,
 <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.900001>)

In [9]:
H = tf.constant((1/4)**0.5)

In [10]:
H

<tf.Tensor: shape=(), dtype=float32, numpy=0.5>

In [187]:
x = tf.Variable([2, -2, 0.5, -0.5])
with tf.GradientTape() as tape:
  y = x**2
dy_dx = tape.gradient(y, x)
print(dy_dx)


tf.Tensor([ 4. -4.  1. -1.], shape=(4,), dtype=float32)


In [186]:
opt = tf.keras.optimizers.experimental.SGD(learning_rate=1, clipvalue=1)
opt2 = tf.keras.optimizers.experimental.SGD(learning_rate=1)
var = tf.Variable([1.0, -1.0, 2.0, -2.0])
var2 = tf.Variable([1.0, -1.0, 2.0, -2.0])
loss = lambda: (var ** 2)/2     # d(loss)/d(var1) = var1
loss2 = lambda: (var2 **2)/2
opt.minimize(loss, [var])
opt2.minimize(loss2, [var2])
var.numpy(), var2.numpy()


(array([ 0.,  0.,  1., -1.], dtype=float32),
 array([0., 0., 0., 0.], dtype=float32))

In [183]:
[ 0.,  0.,  1., -1.]
[0., 0., 0., 0.]

True

In [182]:
np.all(var.numpy()[:2] == var2.numpy()[:2])

True

In [181]:
np.any(var.numpy()[2:] == var2.numpy()[2:])

False

In [ ]:
np.all(var.numpy()[:2] == var.numpy()[2:] + np.array([-1, 1]))
np.all(var.numpy()[:2] == var2.numpy()[:2])
np.any(var.numpy()[2:] == var2.numpy()[2:])

In [224]:
t = np.array([-1.5, -0.5, 0.5, 1.5])#array([-1. , -0.5,  0.5,  1. ])
larq.constraints.weight_clip(clip_value=1)(t).numpy()

array([-1. , -0.5,  0.5,  1. ])

In [216]:
b = BinaryLSTM.BinaryLSTM(8,input_shape=(5,1), return_sequences=False, 
                                 recurrent_constraint=larq.constraints.weight_clip(clip_value=1), 
                                 kernel_constraint=larq.constraints.weight_clip(clip_value=1), 
                                 kernel_initializer=tf.keras.initializers.RandomNormal(mean=3.0, stddev=0.05, seed=None))

In [217]:
b.build((5,1))

In [218]:
b.get_weights()[0]

array([[3.006799 , 2.9681466, 2.9501028, 3.0037122, 2.9788074, 2.967733 ,
        3.055411 , 2.973144 , 2.9833627, 3.0263116, 3.0011606, 2.9910867,
        3.060107 , 2.9705834, 3.020982 , 2.9963572, 3.1093817, 3.0233538,
        3.080354 , 3.0045936, 2.9796624, 2.9583998, 2.9868903, 2.9961188,
        2.9358938, 3.0377724, 2.9603631, 2.975624 , 2.9224513, 2.9904757,
        2.978702 , 2.9903111],
       [3.037035 , 2.8892589, 2.930373 , 3.0734606, 2.9530005, 2.9746168,
        2.9592485, 3.0105615, 2.9320374, 3.012947 , 2.9929729, 3.0057836,
        2.9778128, 3.0284352, 2.946701 , 2.997408 , 2.9963315, 3.029948 ,
        2.9998133, 3.0245006, 3.0260544, 3.0086467, 2.9706426, 3.086245 ,
        3.0991108, 3.0699747, 2.911181 , 2.896552 , 2.9372873, 3.012687 ,
        2.986786 , 3.0051823],
       [3.0922165, 3.0620704, 3.0059662, 2.9990418, 3.080154 , 3.1378572,
        2.935661 , 3.0218713, 3.0490148, 2.9967926, 3.040604 , 3.0489383,
        2.9836211, 2.8785315, 2.999597 , 2.9718244

In [219]:
b.get_weights()[1]

array([[-0.33662844,  0.02268967, -0.10870876,  0.00138171,  0.43672714,
         0.07335119,  0.21218398,  0.00235464, -0.09156472,  0.21831538,
         0.06054892,  0.15698169,  0.01939154,  0.21984376,  0.16891044,
         0.11506853,  0.0780934 ,  0.01028428,  0.01737587, -0.01846097,
        -0.13452296,  0.05120124, -0.1068567 ,  0.19481085, -0.13973798,
         0.24555379, -0.13148066,  0.06379608,  0.07197721,  0.1329926 ,
         0.05970582,  0.5038124 ],
       [ 0.25341275, -0.08038482,  0.31657696,  0.03680122,  0.11060476,
         0.04890069, -0.1438376 ,  0.13537078, -0.08510467,  0.19864634,
         0.12096954, -0.01498451, -0.0807754 ,  0.15261248, -0.09916177,
        -0.17024776, -0.2075933 ,  0.38442063, -0.05192041,  0.34290254,
         0.20675823,  0.02236089,  0.2929447 , -0.05456093, -0.214019  ,
         0.0963771 ,  0.09230691, -0.04378972,  0.07761604,  0.33753145,
        -0.07009356,  0.10159596],
       [-0.21902144, -0.25461772,  0.10311641,  0.1045

In [220]:
b.get_weights()[2]

array([0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

In [221]:
b.get_weights()[3]

array([-0.0259705 ,  0.01829984,  0.00878069,  0.03420207, -0.08326708,
       -0.10949581, -0.06548138, -0.0442927 ,  0.00456281,  0.08912504,
       -0.00442813, -0.07862828,  0.00186113,  0.10798999,  0.10087901,
        0.08036437, -0.05352107, -0.0172999 , -0.00377737,  0.0003107 ,
        0.02393694,  0.04265284,  0.04541467, -0.01326547,  0.06633536,
       -0.01025995,  0.04990179,  0.05589107, -0.00743521,  0.03404764,
        0.011048  ,  0.02461924], dtype=float32)

In [ ]:
model2 = Sequential()
model2.add(BinaryLSTM.BinaryLSTM(8,input_shape=(5,1), return_sequences=False, 
                                 recurrent_constraint=larq.constraints.weight_clip(clip_value=1), 
                                 kernel_constraint=larq.constraints.weight_clip(clip_value=1)))
model2.add(Dense(2,kernel_initializer='normal', activation='linear'))
model2.add(Dense(1,kernel_initializer='normal', activation='linear'))
opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model2.compile(loss='mse',optimizer = opt, metrics=['MSE'])
model2.fit(X, y, epochs=2000, batch_size=5, validation_split=0.05, verbose=0)